In [0]:
dbutils.secrets.list(scope='roshanscope')

Out[1]: [SecretMetadata(key='storagekey'), SecretMetadata(key='storagename')]

In [0]:
spark.conf.set(
  "fs.azure.account.key." + dbutils.secrets.get(scope="roshanscope",key="storagename") + ".dfs.core.windows.net",
  dbutils.secrets.get(scope="roshanscope",key="storagekey"))

In [0]:
filePath = "abfss://roshancontainer@" + dbutils.secrets.get(scope="roshanscope",key="storagename") + ".dfs.core.windows.net/"

In [0]:
dbutils.fs.ls(filePath)

Out[24]: [FileInfo(path='abfss://roshancontainer@[REDACTED].dfs.core.windows.net/finaltitanic.csv/', name='finaltitanic.csv/', size=0, modificationTime=1684947919000),
 FileInfo(path='abfss://roshancontainer@[REDACTED].dfs.core.windows.net/titanic.csv', name='titanic.csv', size=60302, modificationTime=1684944281000)]

In [0]:
df = spark.read.format("csv").option("inferSchema", "true").option("header",
"true").option("delimiter",",").load(filePath+"titanic.csv")

In [0]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [0]:
from pyspark.sql.functions import col
#Task1a.How many people survive and how many people dead 
survived=df.filter(col("Survived")==1).count() 
print("The number of people who survived is",survived)

The number of people who survived is 342


In [0]:
from pyspark.sql.functions import col
died=df.filter(col("Survived")==0).count() 
print("The number of people who died is",died)

The number of people who died is 549


In [0]:
#Task2 How many people survive how does not have sibling
survived_sibling=df.filter((col("Survived")==1) & (col("SibSp") ==0)).count()
print("The number of people who survived and donot have sibling:",survived_sibling)

The number of people who survived and donot have sibling: 210


In [0]:
#Task3 How many people dead how does not have sibling 
dead_nosibling=df.filter((col("Survived")==0) & (col("SibSp") ==0)).count()
print("The number of people who dead and donot have sibling:",dead_nosibling)


The number of people who dead and donot have sibling: 398


In [0]:
#TASK5:How many male who died and how many female died
#for male
male_dead=df.filter((col("Survived")==0) & (col("Sex") =="male")).count()
print("Male who died is",male_dead)
#for female
female_dead=df.filter((col("Survived")==0) & (col("Sex") =="female")).count()
print("Female who died is",female_dead)

Male who died is 468
Female who died is 81


In [0]:
#Task4.Remove the NAN data from cabin and display updated list
df_new=df.na.drop(subset="cabin")
df_new.show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+--------+-----------+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|   PC 17599| 71.2833|        C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|     113803|    53.1|       C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|      17463| 51.8625|        E46|       S|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    1|    PP 9549|    16.7|         G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|     113783|   26.55|       C103|       S|
|         22|       1|     2|Beesley, Mr. Lawr...|  male|34.0|  

In [0]:

data=[( 342,549,210,398,468,81)]
columns=["deadcount","survivedcount","survivednosib","deadnosib","maledead","femaledead"]

df_2= spark.createDataFrame(data,columns)
df_2.show()

+---------+-------------+-------------+---------+--------+----------+
|deadcount|survivedcount|survivednosib|deadnosib|maledead|femaledead|
+---------+-------------+-------------+---------+--------+----------+
|      342|          549|          210|      398|     468|        81|
+---------+-------------+-------------+---------+--------+----------+



In [0]:
# Defining the output path in Azure Data Lake Storage
outputPath = "abfss://roshancontainer@" + dbutils.secrets.get(scope="roshanscope", key="storagename") + ".dfs.core.windows.net/finaltitanic.csv"

# Writing DataFrame to CSV file
df_2.write.format("csv").option("header", "true").mode("overwrite").option("delimiter", ",").save(outputPath)

In [0]:
filePath1 = "abfss://roshancontainer@" + dbutils.secrets.get(scope="roshanscope",key="storagename") + ".dfs.core.windows.net/finaltitanic.csv"
df = spark.read.format("csv").option("inferSchema", "true").option("header",
"true").option("delimiter",",").load(filePath1)

In [0]:
df.show()

+---------+-------------+-------------+---------+--------+----------+
|deadcount|survivedcount|survivednosib|deadnosib|maledead|femaledead|
+---------+-------------+-------------+---------+--------+----------+
|      342|          549|          210|      398|     468|        81|
+---------+-------------+-------------+---------+--------+----------+

